# Getting 48h weather forecast and past 7d weather from NWS via webscraping

This script is intended to collect data from the National Weather Service (NWS) into a **_pandas_** dataframe and to create a simple overview figure using **_matplotlib_**. I am later planning to run this script periodically to create an updated figure. 

Data sources: 
1. [NWS Metar Reports](https://www.wrh.noaa.gov/zoa/getobext.php?sid=KCHO)
2. [NWS Hourly Forecast](https://forecast.weather.gov/MapClick.php?lat=38.1386&lon=-78.4528&lg=english&&FcstType=digital)

In [87]:
import requests
import sys
import pandas as pd
import re 
from bs4 import BeautifulSoup


Select Station using NWS Site Code 

In [88]:
SiteID = "KCHO"
ReportSite = 'https://www.wrh.noaa.gov/zoa/getobext.php?sid=' + SiteID 


In [89]:
TimeNow = pd.to_datetime('today')
YearNow = TimeNow.year


# Define functions to scape data

In [90]:
# Define function to scrape met report 
def ScrapeMetReport(Page):
    "Scrape MetReport Data from NWS for Site with ID and return df(data), str(time of observation), and (coordinates)"
    soup = BeautifulSoup(Page.content, 'html.parser')
    # Extract data from table body
    table_body = soup.find('table',class_="inner-timeseries")
    rows = table_body.find_all('tr')
    tabs=[]
    HeaderLines = 3
    ColumnNames1 = ['Time', 'Temperature', 'Dewpoint', 'Relative Humidity', 'Wind Dir', 'Surface Wind', 'Visibility', 'WX', 'Clouds', 'SLP', 'Altimeter',
               'StationP', '6h TMAX', '6h TMIN', '24h TMAX', '24h TMIN','QC']
    ColumnNames2 = ['Time', 'Temperature', 'Dewpoint', 'Relative Humidity', 'Wind Dir', 'Surface Wind', 'Visibility', 'WX', 'Clouds', 'SLP', 'Altimeter',
               'StationP', 'Rain','P3h','P6h','P24h','6h TMAX', '6h TMIN', '24h TMAX', '24h TMIN','QC']

    for row in rows[HeaderLines:]:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        tabs.append(cols)
    # and write data to dataframe
    if (len(cols)==17):
        df = pd.DataFrame(tabs, columns=ColumnNames1) 
    elif (len(cols) == 21):
        df = pd.DataFrame(tabs, columns=ColumnNames2) 
    else:
        raise Exception('Unexpected number of data columns MetReport')
        
    # Adjust to proper date
    YDiff = YearNow-1900
    df['Time'] = pd.to_datetime(df['Time'], format='%d %b %H:%M %p')
    df['Time'] = df['Time'].apply(lambda x: x + pd.DateOffset(years=YDiff))
    
    # Extract time of observations from website
    table_body = soup.find_all('table')
    rows = table_body[1].find_all('tr')
    cols=rows[2].find_all('td')
    cols=rows[3].find_all('td')
    TimeObs = cols[1].get_text()
    
    # Extract Coordinates 
    LatLonStr = table_body[1].find_all(text=re.compile("Latitude"))
    LatLonStr = re.split(' |;',LatLonStr[0] )
    Coord = tuple([w for w in LatLonStr if re.search('-?[0-9]{1,3}(?:\.[0-9]{1,10})', w)])

    return df, TimeObs, Coord 
    

In [91]:
# Define function to scrape met forecast
def ScrapeForecast(Page):
    "Scrape 2-day forecast from NWS for Site and return df(data)"
    soup = BeautifulSoup(ForecastPage.content, 'html.parser')
    # The site is a bit of a mess, structure wise. Simply get all table rows on site 
    rows = soup.find_all('tr')
    tabs = []
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        tabs.append(cols)
    
    # Select data by length of columns and get column names
    ColNames = [item[0].split('(')[0].strip() for item in tabs if len(item)==25]
    Data = [item[1:] for item in tabs if len(item)==25]

    # Parse data into dataframe 
    df1 = pd.DataFrame(Data[:int(len(Data)/2)]).transpose()
    df1.columns = ColNames[:int(len(Data)/2)]
    df2 = pd.DataFrame(Data[int(len(Data)/2):]).transpose()
    df2.columns = ColNames[int(len(Data)/2):]        
    df = pd.concat([df1, df2])

    # assemble date 
    df['Year']  = YearNow
    df['Month'], df['Day'] = df['Date'].str.split('/', 1).str
    df['Month']=pd.to_numeric(df['Month'])
    df['Month']=df['Month'].fillna(method='ffill')
    df['Day']=df['Day'].fillna(method='ffill')
    df['Date'] = pd.to_datetime(dict(year=df['Year'], month=df['Month'], day=df['Day'], hour=df['Hour']))
    df = df.rename(columns={'Date': 'Time'})
    df.drop(['Year','Month','Day','Hour'], axis=1, inplace=True)
    
    return df

# Main body: Execute the code and return dataframes

In [97]:
try:
    # Open page and parse to soup
    ReportPage = requests.get(ReportSite)
    RepDf, TimeObs, SiteCoord =ScrapeMetReport(ReportPage)
except:
    print('Failed to read MetReport - Something went wrong')
    sys.exit()
    
#RepDf 

In [93]:
# Assemble webpage of forcast for site location
ForecastSite = 'https://forecast.weather.gov/MapClick.php?lat={:6.4f}&lon={:6.4f}&lg=english&&FcstType=digital'.format(float(SiteCoord[0]),float(SiteCoord[1]))
#ForecastSite

'https://forecast.weather.gov/MapClick.php?lat=38.1374&lon=-78.4552&lg=english&&FcstType=digital'

In [98]:
try:
    # Open page and parse to soup
    ForecastPage = requests.get(ForecastSite)
    ForecastDf =ScrapeForecast(ForecastPage)
except:
    print('Failed to read Forecast - Something went wrong')
    sys.exit()
    
#ForecastDf 

# To Do ... Plotting 

In [ ]:
# work in progress 